<h2>Term Implementations</h2>

In [22]:
#deepcopy used to create repeated terms while keeping the original (that is possibly used elsewhere) intact.
from copy import deepcopy

In [23]:
#Implementation of contant. Overrides str, len, eq, and hash.
class Constant:
    registry = set()
    
    def __init__(self, s):
        self.sym = s
        self.arity = 0
        self.length = 1
        Constant.registry.add(self)
        
    def __str__(self):
        return self.sym
        
    def __len__(self):
        return self.length
    
    def __eq__(self, other):
        return isinstance(other, Constant) and self.sym == other.sym

    def __hash__(self):
        return id(str(self))
    
    #Tested different naming scheme. Not currently in use.
    def name(self,s):
        if sum([1 if i.sym==s else 0 for i in Constant.registry])>0:
            return self.name(s + ".")
        else: return s
    
    #Constant size can not be reduced. Higher-level structures can determine if it is a repeat.
    def reduceSize(self):
        pass
    
    #Will always be a ground term. Base of higher level isGround recursion.
    def isGround(self):
        return True
    
    #No variables. Returns empty list to conform with other structures use of list concacts.
    def findVariables(self):
        return []
    
    #Is Inherently a repeated term once found. Base of higher level findRepeats recursion.
    def findRepeats(self):
        return [self]
    
    #Nothing to replace. Base of higher level replace recursion.
    def replace(self, var, rpt):
        return self
    
print "Constant structure loaded."

Constant structure loaded.


In [24]:
#Implementation of repeat. Encapsulates replaced object in case the information is needed. Length is always 0.
#Overrides str, len, eq, and hash
class Repeat:
    def __init__(self, original):
        self.sym = '*'
        self.origin = original
        self.length = 0
    
    #Currently shows just a star. Switch commented line for more informative printed information.
    def __str__(self):
        #return self.sym + str(self.origin) + self.sym
        return self.sym
        
    def __len__(self):
        return self.length
    
    #Checks encapsulated object of equality beyond just being a repeat.
    def __eq__(self, other):
        return isinstance(other, type(self.origin)) and \
        self.origin.sym == other.sym and self.origin.params == other.params
        
    def __hash__(self):
        return id(str(self.origin))
    
    #Size already reduced. Implemented so that reduceSize still works if accidentally called multiple times.
    def reduceSize(self):
        pass
    
    #Ground determined by encapsulated object. Passed on to that function.
    def isGround(self):
        return self.origin.isGround()
    
    #Variables determined by encapsulated object. Passed on to that function.
    def findVariables(self):
        return self.origin.findVariables()
    
    #First instance of this repeat already included in search. 
    #Empty list used to conform with higher level findRepeat.
    def findRepeats(self):
        return []
    
    #No variable to replace. Should not replace underlying object (if possible) 
    #because there is some clause that does not have this as a repeat.
    def replace(self, var, rpt):
        return self

print "Repeat structure loaded."

Repeat structure loaded.


In [25]:
#Implementaion of variable. Frequently replaced with other objects in enumerate algorithm.
#Overrides str, len, eq, and hash.
class Variable:
    registry = set()
    
    def __init__(self, s):
        self.sym = s
        self.length = 1 #Use minimal possible size.
        #self.length = 0
        Variable.registry.add(self)
        
    def __str__(self):
        return self.sym
    
    def __len__(self):
        return self.length
    
    def __eq__(self, other):
        return isinstance(other, Variable) and self.sym == other.sym
    
    def __hash__(self):
        return id(str(self))
    
    #Previous naming scheme testing. Not in use.
    def name(self, s):
        if sum([1 if i.sym==s else 0 for i in Variable.registry])>0:
            return self.name(s + ".")
        else: return s
    
    #Variable size can not be reduced. Higher-level structures can determine if it is a repeat.
    def reduceSize(self):
        pass
    
    #Will never be a ground term. Base of higher level isGround recursion.
    def isGround(self):
        return False
    #Is a variable. Returns object wrapped in list to conform with other structures use of list concacts.
    #Base of higher level findVariables recursion.
    def findVariables(self):
        return [self]
    
    #Is Inherently a repeated term once found. Base of higher level findRepeats recursion.
    def findRepeats(self):
        return [self]
    
    def replace(self, var, rpt):
        variable = deepcopy(self)
        if self==var:
                return rpt
        else:
                return

print "Variable structure loaded."

Variable structure loaded.


In [26]:
#Implementation of function. Includes symbol (must be unique) and list of paramaters.
#Registry containts copies of all functions that may be altered in enumerate algorithm.
#Consider including arity parameter.
#Overrides str, len, eq, and hash
class Function:
    
    #Length calculated by definition: 1 + length of each parameter.
    def __init__(self, s, *args):
        self.sym = s
        self.params = list(args)
        self.arity = len(args)
        self.length = 1+sum([len(i) for i in self.params])

    def __str__(self):
        toReturn = self.sym + "("
        for i in self.params:
            toReturn += str(i) + ","
        toReturn = toReturn[:-1] + ")"
        return toReturn
    
    def __len__(self):
        return 1+sum([len(i) for i in self.params])
                     
    def __eq__(self, other):
        return isinstance(other, Function) and self.sym == other.sym and self.params == other.params
    
    def __hash__(self):
        return id(str(self))
    
    #Previous naming scheme testing. Not in use.
    def name(self,s):
        if sum([1 if i.sym==s else 0 for i in Function.registry])>0:
            return self.name(s + ".")
        else: return s
    
    #For each parameter (0 to arity), scans forward for repeated terms. Any repeats are replaced by Repeats.
    #Then reduces size of each parameter.
    def reduceSize(self):
        for i in xrange(len(self.params)):
            for j in xrange(i+1, len(self.params)):
                if not isinstance(self.params[j], Repeat) and self.params[i] == self.params[j]:
                    self.params[j] = Repeat(self.params[j])
        for i in self.params:
            i.reduceSize()
    
    #Scans each parameter. If any are not ground, the entire instance of the function is not ground.
    def isGround(self):
        for i in self.params:
            if not i.isGround():
                return False
        return True
    
    #Scans each parameter. Finds variables in each parameter and adds all of them to the list.
    def findVariables(self):
        variables = []
        for i in self.params:
            variables = variables + i.findVariables()
        return variables
    
    #Scans each parameter. Finds repeats in each parameter and adds all of them to the list.
    def findRepeats(self):
        repeats = []
        for i in self.params:
            repeats = repeats + i.findRepeats()
        repeats.append(self)
        return repeats
    
    def replace(self, var, rpt):
        function = deepcopy(self)
        for i in xrange(len(function.params)):
            term = function.params[i]
            if term==var:
                function.params[i] = rpt
            else:
                term.replace(var, rpt)
        return function

print "Function structure loaded."

Function structure loaded.


In [27]:
#Implementation of predicate (very similar implementation to function). 
#Includes symbol (must be unique) and list of paramaters.
#Registry containts copies of all functions that may be altered in enumerate algorithm.
#Overrides str, len, eq, and hash
class Predicate:
    
    #Length calculated by definition: 1 + length of each parameter.
    def __init__(self, s, *args):
        self.sym = s
        self.params = list(args)
        self.length = 1+sum([len(i) for i in self.params])
       
    def __str__(self):
        toReturn = self.sym + "("
        for i in self.params:
            toReturn += str(i) + ","
        toReturn = toReturn[:-1] + ")"
        return toReturn
    
    def __len__(self):
        return 1+sum([len(i) for i in self.params])
    
    def __eq__(self, other):
        return isinstance(other, Predicate) and self.params == other.params
 
    def __hash__(self):
        return id(self.sym)+1
    
    #Previous naming scheme testing. Not in use.
    def name(self,s):
        if sum([1 if i.sym==s else 0 for i in Function.registry])>0:
            return self.name(s + ".")
        else: return s
    
    #For each parameter (0 to arity), scans forward for repeated terms. Any repeats are replaced by Repeats.
    #Then reduces size of each parameter.
    def reduceSize(self):
        for i in xrange(len(self.params)):
            for j in xrange(i+1, len(self.params)):
                if not isinstance(self.params[j], Repeat) and self.params[i] == self.params[j]:
                    self.params[j] = Repeat(self.params[j])
        for i in self.params:
            i.reduceSize()
            
    #Scans each parameter. If any are not ground, the entire instance of the function is not ground.
    def isGround(self):
        for i in self.params:
            if not i.isGround():
                return False
        return True
    
    #Scans each parameter. Finds variables in each parameter and adds all of them to the list.
    def findVariables(self):
        variables = []
        for i in self.params:
            variables = variables + i.findVariables()
        return variables
    
    #Scans each parameter. Finds repeats in each parameter and adds all of them to the list.
    def findRepeats(self):
        repeats = []
        for i in self.params:
            repeats = repeats + i.findRepeats()
        repeats.append(self)
        return repeats

    def replace(self, var, rpt):
        predicate = deepcopy(self)
        for i in xrange(len(predicate.params)):
            term = predicate.params[i]
            if term==var:
                predicate.params[i] = rpt
            else:
                predicate.params[i] = term.replace(var, rpt)
        return predicate
    
print "Predicate structure loaded."

Predicate structure loaded.


In [39]:
#Implementation of clause (very similar implementation to function and predicate). 
#Includes list of paramaters.
#Registry containts copies of all functions that may be altered in enumerate algorithm.
#Overrides str, len, eq, and hash
class Clause: 
    
    #Length calculated by definition: sum of length of each parameter.
    def __init__(self, *args):
        self.params = list(args)
        self.length = sum([len(i) for i in self.params])
        
    def __str__(self):
        toReturn = "{"
        for i in self.params:
            toReturn += str(i) + ","
        toReturn = toReturn[:-1] + "}"
        return toReturn
    
    def __len__(self):
        return sum([len(i) for i in self.params])
    
    def __eq__(self, other):
        return isinstance(other, Function) and self.sym == other.sym and self.params == other.params

    def __hash__(self):
        return id(str(self))
    
    #For each parameter (0 to arity), scans forward for repeated terms. Any repeats are replaced by Repeats.
    #Then reduces size of each parameter. Returns new modified clause.
    def reduceSize(self):
        for i in xrange(len(self.params)):
            for j in xrange(i+1, len(self.params)):
                if not isinstance(self.params[j], Repeat) and self.params[i] == self.params[j]:
                    self.params[j] = Repeat(self.params[j])
        for i in self.params:
            i.reduceSize()    
        return self
    
    #Scans each parameter. If any are not ground, the entire instance of the function is not ground.
    def isGround(self):
        for i in self.params:
            if not i.isGround():
                return False
        return True
    
    #Scans each parameter. Finds variables in each parameter and adds all of them to the list.
    def findVariables(self):
        variables = []
        for i in self.params:
            variables = variables + i.findVariables()
        return set(variables)
    
    #Scans each parameter. Finds repeats in each parameter and adds all of them to the list.
    def findRepeats(self):
        repeats = []
        for i in self.params:
            repeats = repeats + i.findRepeats()
        repeats.append(self)
        return set(repeats)
    
    def replace(self, var, rpt):
        clause = deepcopy(self)
        for i in xrange(len(clause.params)):
            term = clause.params[i]
            if term==var:
                clause.params[i] = rpt
            else:
                clause.params[i] = term.replace(var, rpt)
        return clause
    
print "Clause structure loaded."

Clause structure loaded.


In [41]:
#All constants, variables, functions, and predicates MUST be uniquely named.

SIZE_BOUND = 3

#Input: Clause and sizebound. Constants, variables, functions, and predicates within the clause MUST be uniquely named.
#Output: Terms in Herbrand universe whose modified size is less than the sizebound.
def enumerateClauses(clause, size, rpts = set(), clauseSet = set()):
    repeats = rpts.union(clause.findRepeats())
    if size<0:
        return
    elif size==0:
        x = None
        for x in clause.findVariables():
            for i in repeats:
                clauseSet.add(deepcopy(clause).replace(x,i))
        if x==None:
            clauseSet.add(clause)
    else:
        for x in clause.findVariables():
            for i in set(Constant.registry).union(set(Function.registry)).union(set(repeats)): ## THE PROBLEM
                if (size-len(i))>=0:
                    enumerateClauses(deepcopy(clause).replace(x,i), size-len(i), repeats, clauseSet)
        sizeCheck(SIZE_BOUND, repeats, clauseSet)
    return clauseSet
        
def sizeCheck(size, repeats, clauseSet):
    diff = set()
    for clause in clauseSet:
        if len(clause.reduceSize()) > size:
            diff.add(clause)
        else:
            for i in clause.params:
                repeats.add(i)
    clauseSet = clauseSet-diff

<h2>Testing Zone</h2>
Replace is not currently working because of the way listcomps work in python, Cant change values in the list using a 'for i in list' approach. Working on fix.

In [40]:
a = Constant('a')
b = Constant('a')
x = Variable('x')
y = Variable('y')
f1 = Function('f', a, x)
f2 = Function('f', a, a)
g1 = Function('g', b, x)
g2 = Function('g', b, y)
g3 = Function('g', b, y)
p1 = Predicate('P',f1, f2, g2, g3)
p2 = Predicate('Q', f1)
p3 = Predicate('R', f2, g3)
c1 = Clause(p1, p2, p3)
c2 = Clause(f1)

"""
print "p1 before"
for i in p1.params:
    print i
    
pt = deepcopy(p1)
pt.params[0] = a

print "p1 after"
for i in p1.params:
    print i

print "pt now"
for i in pt.params:
    print i
    

for i in c1.findRepeats():
    print i

c3 = Clause(x)  
x = None
for x in c3.findVariables():
    print x
print x

a = Constant('a')
b = Constant('b')
x = Variable('x')
f1 = Function('f', a, x)
c2 = Clause(f1)   

for i in enumerateClauses(c2,SIZE_BOUND):
    print i
    
"""

for x in c1.findVariables():
    print x

x
y
